In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, savgol_filter
from scipy.signal import peak_widths
from matplotlib.gridspec import GridSpec
import csv
import os
import ipympl
import pprint

class LaserData:
    def __init__(self,path):
        self.wavelenghts = np.array([229,233,237,241,245,249,253]) +780
        with open(path) as file:
            self.file = file
            self.file = csv.reader(self.file, delimiter='\t')
            self.time =[]
            self.piezo_voltage = []
            self.fabry_perot = []
            self.probe_laser = []
            self.laser_voltage = []
            for line in self.file:
                self.time.append(float(line[0]))
                self.piezo_voltage.append(float(line[1]))
                self.fabry_perot.append(float(line[2]))
                self.probe_laser.append(float(line[3]))
                self.laser_voltage.append(float(line[4]))
            self.time = np.array(self.time)
            self.piezo_voltage = np.array(self.piezo_voltage)
            self.fabry_perot = np.array(self.fabry_perot)
            self.probe_laser = np.array(self.probe_laser)
    
    def plot(self,with_peaks = False):
        fig = plt.figure(figsize=(11,6))
        gs = GridSpec(8,5)
        fig1 = fig.add_subplot(gs[:,:])
        fig1.plot(self.piezo_voltage,self.fabry_perot,'-')
        #fig1.plot(self.time,self.piezo_voltage)
        if with_peaks:
            peaks,_ = find_peaks(self.fabry_perot,prominence=0.3,height=0.8)
            print(len(peaks))
            for peak in peaks:
                fig1.plot(self.piezo_voltage[peak],self.fabry_perot[peak],'go')
        fig1.set_xlabel('Zeit in s',fontsize = 15)
        fig1.set_ylabel('Spannung in V',fontsize = 15)

    def slice_data(self, idx_start, idx_stop):
        # deletes the data outside the given index_range
        self.time = self.time[idx_start:idx_stop] - self.time[idx_start]
        self.piezo_voltage = self.piezo_voltage[idx_start:idx_stop]
        self.fabry_perot = self.fabry_perot[idx_start:idx_stop]
        self.probe_laser = self.probe_laser[idx_start:idx_stop]
        self.laser_voltage = self.laser_voltage[idx_start:idx_stop]

    

laser = LaserData('Data/spektrum_dopplerfrei_mit_pump_freq4.dat') 

In [ ]:
laser.plot(with_peaks=True)

# Nicht dopplerfrei

In [ ]:
#relevante files einlesen
filenames = sorted([file for file in os.listdir('Data/') if file.__contains__('nicht_dopplerfrei_freq')])

#new figure
%matplotlib widget
#plt.clf()
fig, axes = plt.subplots(1, 2, figsize=(11,6))

#Piezo-Spannungen für die händisch gemessenen Peaks (für absteigende Spannung)
peaks_manuell_V = np.array([-13.2, -8.36, -2.67, 2.96, 8.70, 14.6])

#korrespondierende Wellenlängen
wavelengths_nm = np.array([229, 233, 237, 241, 245, 249]) * 10**(-3) + 780

for file in filenames:
    laser = LaserData('Data/'+file)

    #starting time
    idx_start = np.argmax(laser.piezo_voltage)
    idx_stop  = np.argmin(laser.piezo_voltage)

    t_start = laser.time[idx_start]
    t_stop = laser.time[idx_stop]

    period = 2 * (t_stop - t_start)

    
    laser.slice_data(idx_start, idx_stop)
    #plt.plot(laser.piezo_voltage)
    laser.piezo_voltage = savgol_filter(laser.piezo_voltage, 1000, 1)
    
    peaks,_ = find_peaks(laser.fabry_perot,prominence=0.3,height=0.8)
    

    ax = axes[0]
    ax.plot(5 * laser.piezo_voltage, laser.fabry_perot, label=f'T={period:.3f} s')
    for peak in peaks:
        ax.plot(5 * laser.piezo_voltage[peak], laser.fabry_perot[peak],'go')

    for i, peak in enumerate(peaks_manuell_V):
        ax.plot(np.array([peak, peak]), [0.2, 1.4],'--', color='black', linewidth=1)
    
    ax = axes[1]

    
    ax.scatter(5 * np.sort(laser.piezo_voltage[peaks])[::-1], wavelengths_nm[1::], marker='x', label=f'T={period:.3f} s')


    pprint.pprint(5 * np.sort(laser.piezo_voltage[peaks]))
    pprint.pprint(wavelengths_nm)

    ax.legend()
    ax.set_xlabel('Piezospannung in V')
    ax.set_ylabel('Fabry-Perot-Spannung in V')
    #laser.plot(with_peaks=True)


# Dopplerfrei